In [14]:
%%capture
!pip install ultralytics

In [15]:
!pip install wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 41.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [wandb]32m6/7 [wandb]hon]]


In [16]:
import os
os.environ["WANDB_DISABLE_SSL"] = "true"
os.environ["ALBUMENTATIONS_DISABLE"] = "1" # Must be set before YOLO import
from ultralytics import settings

settings.update({"wandb": True,
                 "clearml": False,
                 "comet": False})

In [17]:
import torch

def get_train_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("Using NVIDIA GPU (CUDA)")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Using Apple Silicon GPU (MPS)")
    else:
        device = torch.device("cpu")
        print("Using CPU")
    return device

device_for_training = get_train_device()

Using Apple Silicon GPU (MPS)


In [ ]:
from ultralytics import YOLO
from ultralytics import settings
import wandb
wandb.init(project="odir-2019", name="experiment-1")

model = YOLO("yolo11n-cls.pt")  # Load a pre-trained YOLO model
result = model.train(data="ODIR-2019/YOLO/preprocessed",
                     epochs=1,
                     save_period=1,
                     batch=-1,
                     device=device_for_training,
                     project='odir-2019',
                     save_json=True,  # Saves metrics to JSON file
                     verbose=True,
                     plots=True)
wandb.finish()

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /Users/Ray/.netrc.
wandb: Currently logged in as: raymond-samalo (samalo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=ODIR-2019/YOLO/preprocessed, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=odir-2019, rect=False, resu

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


In [23]:
metrics=model.val(data="ODIR-2019/YOLO/preprocessed")
precision = metrics.precision  # Mean precision
recall = metrics.recall        # Mean recall

# 4. Calculate F1 Score
if (precision + recall) > 0:
    f1_score = 2 * (precision * recall) / (precision + recall)
    print(f"Mean F1 Score: {f1_score:.4f}")
else:
    print("F1 Score cannot be calculated (denominator is zero)")

Ultralytics 8.4.7 🚀 Python-3.14.2 torch-2.9.1 CPU (Apple M1)
YOLO11n-cls summary (fused): 47 layers, 1,536,272 parameters, 0 gradients, 3.2 GFLOPs
train: /Users/Ray/Projects/ITI123-GenerativeAIDeepLearningProject/ODIR-2019/YOLO/preprocessed/train... found 5014 images in 8 classes ✅ 
val: /Users/Ray/Projects/ITI123-GenerativeAIDeepLearningProject/ODIR-2019/YOLO/preprocessed/val... found 627 images in 8 classes ✅ 
test: /Users/Ray/Projects/ITI123-GenerativeAIDeepLearningProject/ODIR-2019/YOLO/preprocessed/test... found 627 images in 8 classes ✅ 
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 204.4±79.9 MB/s, size: 55.0 KB)
val: Scanning /Users/Ray/Projects/ITI123-GenerativeAIDeepLearningProject/ODIR-2019/YOLO/preprocessed/val... 627 images, 0 corrupt: 100% ━━━━━━━━━━━━ 627/627 79.7Mit/s 0.0s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 40/40 2.7it/s 14.7s0.4s
                   all      0.482      0.947
Speed: 0.0ms preprocess, 18.5ms inference, 0.0ms loss, 0.0ms po

AttributeError: 'ClassifyMetrics' object has no attribute 'precision'. See valid attributes below.
Class for computing classification metrics including top-1 and top-5 accuracy.

Attributes:
    top1 (float): The top-1 accuracy.
    top5 (float): The top-5 accuracy.
    speed (dict): A dictionary containing the time taken for each step in the pipeline.
    task (str): The task type, set to 'classify'.

Methods:
    process: Process target classes and predicted classes to compute metrics.
    fitness: Return mean of top-1 and top-5 accuracies as fitness score.
    results_dict: Return a dictionary with model's performance metrics and fitness score.
    keys: Return a list of keys for the results_dict property.
    curves: Return a list of curves for accessing specific metrics curves.
    curves_results: Provide a list of computed performance metrics and statistics.
    summary: Generate a single-row summary of classification metrics (Top-1 and Top-5 accuracy).
